In [ ]:
!git clone https://github.com/openclimatefix/metnet.git

fatal: destination path 'metnet' already exists and is not an empty directory.


In [ ]:
%cd metnet

/content/metnet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r requirements.txt
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/metnet
  Attempting uninstall: metnet
    Found existing installation: metnet 4.1.8
    Can't uninstall 'metnet'. No files were found to uninstall.
  Running setup.py develop for metnet


In [ ]:
from metnet import MetNet, MetNet2
model = MetNet().from_pretrained("openclimatefix/metnet")
model = MetNet2().from_pretrained("openclimatefix/metnet-2")

from metnet import MetNet
import torch
import torch.nn.functional as F

model = MetNet(
        hidden_dim=32,
        forecast_steps=24,
        input_channels=16,
        output_channels=12,
        sat_channels=12,
        input_size=32,
        )

# MetNet expects original HxW to be 4x the input size
x = torch.randn((2, 12, 16, 128, 128))
out = []
for lead_time in range(24):
        out.append(model(x, lead_time))
out = torch.stack(out, dim=1)
# MetNet creates predictions for the center 1/4th
y = torch.randn((2, 24, 12, 8, 8))
F.mse_loss(out, y).backward()

KeyboardInterrupt: ignored

In [ ]:
print(out.size())

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import h5py
import matplotlib.pyplot as plt  
from PIL import Image 
from google.colab.patches import cv2_imshow


val_rates = h5py.File('/content/drive/Shareddrives/CSCI5527Project/data/boxi_0034.val.rates.crop.h5', 'r')

print(val_rates.keys())
val_gt = val_rates['rates.crop']
print(val_gt.shape)
#print(val_gt[1].shape)

# img0 = imgs[1]
# img0 = np.squeeze(img0)
# plt.imshow(img0)

val_data = h5py.File("/content/drive/Shareddrives/CSCI5527Project/data/boxi_0034.val.reflbt0.ns.h5")
print(val_data.keys())
val_imgs = val_data['REFL-BT']
print(val_imgs.shape)

<KeysViewHDF5 ['rates.crop']>
(2160, 1, 252, 252)
<KeysViewHDF5 ['REFL-BT']>
(2160, 11, 252, 252)


In [ ]:
input = torch.unsqueeze(torch.tensor(val_imgs[0:4]), dim=0)

print(input.shape)

gt = torch.unsqueeze(torch.tensor(val_gt[0:32]), dim=0)
print(gt.shape)


torch.Size([1, 4, 11, 252, 252])
torch.Size([1, 32, 1, 252, 252])


In [ ]:
from metnet import MetNet, MetNet2
model = MetNet().from_pretrained("openclimatefix/metnet")
#model = MetNet2().from_pretrained("openclimatefix/metnet-2")

from metnet import MetNet
import torch
import torch.nn.functional as F

model = MetNet(
        hidden_dim=32,
        forecast_steps=32,
        input_channels=11,
        output_channels=1,
        sat_channels=11,
        input_size=63,                       #input img size?
        #use_preprocessor=False
        ).double()

# MetNet expects original HxW to be 4x the input size
#x = torch.randn((2, 12, 16, 128, 128))                 #[batch size, seq-length, channels, H, W]
print(input.shape)
out = []
for lead_time in range(32):
        out.append(model(input, lead_time))
out = torch.stack(out, dim=1)
# MetNet creates predictions for the center 1/4th
#y = torch.randn((2, 24, 12, 8, 8))

F.mse_loss(out, gt).backward()

torch.Size([1, 4, 11, 252, 252])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Using a target size (torch.Size([1, 32, 1, 252, 252])) that is different to the input size (torch.Size([1, 32, 1, 15, 15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


RuntimeError: ignored

In [ ]:
print(out.size())
print(gt.size())

torch.Size([1, 32, 1, 15, 15])
torch.Size([1, 32, 1, 252, 252])


In [ ]:
import os
import math
import numpy as np 
from PIL import Image
import cv2
from pathlib import Path
import time
import argparse
import logging 


import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()
from google.colab.patches import cv2_imshow

from tqdm.notebook import tqdm
import torch
import torch.nn as nn   #defines pre-existing set of neural networks
import torch.nn.functional as F   #defines set of functions, that pytorch knows how to autodiff
import torch.utils.data as data   
import torch.optim as optim       
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from pathlib import Path

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
set_seed(42)

#ensure all operations are deterministic on GPU for reproducible results
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)


# class DataLoader(Dataset):
#     def __init__(self, in_size, out_size):
#         self.


#     def get_length():


#     def get_item():


In [ ]:
import torch

def condition_time(i=0, size=(5,5), seq_len=3):
    "create one hot encoded time image-layers, i in [1, seq_len]"
    assert i < seq_len
    # times = (
    #     (torch.eye(seq_len, dtype=torch.long, device=x.device)[i])
    #     .unsqueeze(-1)
    #     .unsqueeze(-1)
    # )
    times = (
        (torch.eye(seq_len, dtype=torch.long)[i])
        .unsqueeze(-1)
        .unsqueeze(-1)
    )
    
    #ones = torch.ones(1, *size, dtype=x.dtype, device=x.device)
    ones = torch.ones(1, *size)#, dtype=x.dtype, device=x.device)

    print(f"times = {times}")
    print(f"ones = {ones}")
    res = times * ones
    print(f"res = {res}")
    return times * ones
condition_time()

times = tensor([[[1]],

        [[0]],

        [[0]]])
ones = tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])
res = tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])


tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [3]:
import os
import math
import numpy as np 
from PIL import Image
import cv2
from pathlib import Path
import time
import argparse
import logging 


import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()
from google.colab.patches import cv2_imshow

from tqdm.notebook import tqdm
import torch
import torch.nn as nn   #defines pre-existing set of neural networks
import torch.nn.functional as F   #defines set of functions, that pytorch knows how to autodiff
import torch.utils.data as data   
import torch.optim as optim       
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from pathlib import Path

In [9]:
input = torch.rand(4, 1, 1, 252, 252)
cropped = transforms.functional.crop(input, 62,62,126,126)
# torchvision.transforms.CenterCrop((126,126))
# print(input.size())

print(cropped.size())
output_cropped = F.interpolate(cropped, (1,252,252),mode='nearest-exact')
print(output_cropped.size())
# output = F.interpolate(input, (1, 126, 126), mode='nearest-exact')
# print(output.size())
# # cropped_output = transforms.CenterCrop((252,252))
# cropped_output = transforms.functional.crop(output, 630,630,252,252)
# # print(type(cropped_output))
# # print(torch.unsqueeze(cropped_output,dim=0))
# print(cropped_output.size())



torch.Size([4, 1, 1, 126, 126])
torch.Size([4, 1, 1, 252, 252])
